## Randomforest classifier implementation with pipelines and hyperparameter tuning

In [1]:
import seaborn as sb
df=sb.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [2]:
## feature engineering processes like
### feature scaling 
### outliers
### handling categorical features
### handling missing values
## automating these all entire processes

In [3]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
df['time']=encoder.fit_transform(df['time'])

In [4]:
df['time'].value_counts()

time
0    176
1     68
Name: count, dtype: int64

In [5]:
## dependent and independent features
X=df.drop(labels=['time'],axis=1)
y=df['time']

In [6]:
X.head()

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4


In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)

In [8]:
from sklearn.preprocessing import StandardScaler ### feature scaling 
from sklearn.preprocessing import OneHotEncoder ### handling categorical features
from sklearn.impute import SimpleImputer ### handling missing values
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer 

In [9]:
cat_cols=['sex','smoker','day']
num_cols=['total_bill','tip','size']

In [10]:
num_pipeline=Pipeline(
         steps=[
             ('imputer',SimpleImputer(strategy='median')), ## missing values
             ('scaler',StandardScaler()) ## feature scaling
         ]
)

cat_pipeline=Pipeline(
         steps=[
             ('imputer',SimpleImputer(strategy='most_frequent')), ## missing values
             ('onehotencoder',OneHotEncoder()) ## catergorical to numerical 
         ]
)

In [11]:
preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,num_cols),
    ('cat_pipeline',cat_pipeline,cat_cols)
])

In [12]:
X_train=preprocessor.fit_transform(X_train)
X_test=preprocessor.transform(X_test)


In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [14]:
## automate entire training data
models={
    'Random Forest':RandomForestClassifier(),
    'decision tree': DecisionTreeClassifier(),
    'svc':SVC()

}

In [15]:
from sklearn.metrics import accuracy_score

In [16]:
def model_evaluater(X_train,y_train,X_test,y_test,models):
    
    report={}
    for i in range(len(models)):
        model=list(models.values())[i]
        model.fit(X_train,y_train) ## model training
        
        
        
        y_test_pred=model.predict(X_test) ## predict testing data
        
        test_model_score=accuracy_score(y_test,y_test_pred)
        report[list(models.keys())[i]]=test_model_score
    return report

In [17]:
model_evaluater(X_train,y_train,X_test,y_test,models)

{'Random Forest': 0.9591836734693877,
 'decision tree': 0.9387755102040817,
 'svc': 0.9591836734693877}

In [18]:
classifier=RandomForestClassifier()

In [19]:
from sklearn.model_selection import RandomizedSearchCV 

In [20]:
params={'max_depth':[3,5,10,None],
       'n_estimators':[100,200,300],
       'criterion':['gini','entropy']
       }

In [21]:
cv=RandomizedSearchCV(classifier,param_distributions=params,scoring='accuracy',cv=5,verbose=3)
cv.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=gini, max_depth=5, n_estimators=300;, score=0.974 total time=   0.5s
[CV 2/5] END criterion=gini, max_depth=5, n_estimators=300;, score=0.923 total time=   0.5s
[CV 3/5] END criterion=gini, max_depth=5, n_estimators=300;, score=0.974 total time=   0.5s
[CV 4/5] END criterion=gini, max_depth=5, n_estimators=300;, score=0.923 total time=   0.5s
[CV 5/5] END criterion=gini, max_depth=5, n_estimators=300;, score=0.949 total time=   0.5s
[CV 1/5] END criterion=gini, max_depth=10, n_estimators=100;, score=0.974 total time=   0.2s
[CV 2/5] END criterion=gini, max_depth=10, n_estimators=100;, score=0.923 total time=   0.2s
[CV 3/5] END criterion=gini, max_depth=10, n_estimators=100;, score=1.000 total time=   0.2s
[CV 4/5] END criterion=gini, max_depth=10, n_estimators=100;, score=0.923 total time=   0.2s
[CV 5/5] END criterion=gini, max_depth=10, n_estimators=100;, score=0.923 total time=   0.2s
[CV 1/5] END c

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [22]:
cv.best_params_

{'n_estimators': 300, 'max_depth': 3, 'criterion': 'gini'}